#PhoBert_large

In [ ]:
!pip install transformers==3.5.1
!pip install torch==1.4.0.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 14.6 MB/s 
     |████████████████████████████████| 1.1 MB 64.0 MB/s 
     |████████████████████████████████| 2.9 MB 61.3 MB/s 
     |████████████████████████████████| 880 kB 66.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=3703c52595fc67ac7dc2dcf9aa38875a9b8fb775f212b148915eb73bbc9784ae
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 753.4 MB 7.8 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0+cu113
    Uninstalling torch-1.12.0+cu113:
      Successfully uninstalled torch-1.12.0+cu113
ERROR: pip's dependency resolver does not cur

In [1]:
from transformers import BertTokenizer, BertForPreTraining, BertForQuestionAnswering, BertModel, BertConfig
from transformers import XLMRobertaForQuestionAnswering, XLMRobertaTokenizer
import torch
import torch.nn as nn
from transformers.data.metrics.squad_metrics import compute_predictions_log_probs, compute_predictions_logits, squad_evaluate
from transformers.data.processors.squad import SquadResult, SquadV1Processor, SquadV2Processor, SquadProcessor, SquadExample

In [2]:
!pip install vncorenlp
!wget 'https://github.com/vncorenlp/VnCoreNLP/archive/v1.1.1.zip' -O ./models.$$ && unzip -o ./models.$$ && rm -r ./models.$$.

Defaulting to user installation because normal site-packages is not writeable


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 2.6/2.6 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2643023 sha256=a5e5b07d5f6da032830b1ef814a1eb702d520c9696f4c631f4313128c498f462
  Stored in directory: c:\users\tranq\appdata\local\pip\cache\wheels\ee\fa\2e\0bda25d261226967771a85a65f910fb6de9eabfd2f1a206afb
Successfully built vncorenlp


'wget' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
from vncorenlp import VnCoreNLP
annotator = VnCoreNLP("./VnCoreNLP-1.1.1/VnCoreNLP-1.1.1/VnCoreNLP-1.1.1.jar",port=9000, annotators="wseg,pos,ner,parse", max_heap_size='-Xmx2g') 
def VnCoreNLP1(X_train):
  X_train = annotator.tokenize(X_train)
  p = ''
  for i in range(0, len(X_train)):
    for x in X_train[i]:
      w = ''
      w = w + x + ' '
      p+=w
  p=p.replace(" ,",",")
  p=p.replace(" .",".")
  p=p.replace("( ","(")
  p=p.replace(" )",")")
  p=p.replace(' " ',' "')
  p=p.replace("* ","*")
  p=p.replace(" ;",";")
  p=p.replace(" ?","?")
  p=p.replace(" {","{")
  p=p.replace("{ ","{")
  p=p.replace(" }","}")
  p=p.replace("} ","}")
  p=p.replace(" /","/")
  p=p.replace("/ ","/")
  return p

KeyboardInterrupt: 

In [ ]:
from transformers import AutoModel, AutoTokenizer
from transformers import RobertaForQuestionAnswering, RobertaConfig, WEIGHTS_NAME

phobert = AutoModel.from_pretrained("vinai/phobert-large")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-large")
model = RobertaForQuestionAnswering(phobert.config).from_pretrained("vinai/phobert-large")

Downloading:   0%|          | 0.00/558 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.48G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/895k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.
Some weights of the model checkpoint at vinai/phobert-large were not used when initializing RobertaForQuestionAnswering: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at vinai/phob

In [ ]:
def checkAns(answer, context_text):
  ans=context_text[answer["answer_start"] : answer["answer_start"]+ len(answer["text"])]
  l=ans.replace("_"," ")
  if(l == answer["text"]):
    return ans, answer["answer_start"]
  else:
    firstWord=answer["text"].split(' ')[0]
    lenWord=len(firstWord)
    for i in range(1,8):
      try:
        position_1=answer["answer_start"]-i
        word=context_text[position_1: position_1+ lenWord]
        if(firstWord == word):
          return context_text[position_1: position_1+ len(answer["text"])],position_1
        position_2=answer["answer_start"]+i
        word=context_text[position_2: position_2+ lenWord]
        if(firstWord == word):
          return context_text[position_2: position_2+ len(answer["text"])],position_2
      except:
        return ans, answer["answer_start"]
    return ans, answer["answer_start"]

In [ ]:
def checkTempAns(answer, context_text):
  firstWord=answer["text"].split(' ')[0]
  lenWord=len(firstWord)
  for i in range(1,8):
    try:
      position_1=answer["answer_start"]-i
      word=context_text[position_1: position_1+ lenWord]
      if(firstWord == word):
        return context_text[position_1: position_1+ len(answer["text"])],position_1
      position_2=answer["answer_start"]+i
      word=context_text[position_2: position_2+ lenWord]
      if(firstWord == word):
        return context_text[position_2: position_2+ len(answer["text"])],position_2
    except:
      return context_text[answer["answer_start"] : answer["answer_start"]+ len(answer["text"])], answer["answer_start"]
  return context_text[answer["answer_start"] : answer["answer_start"]+ len(answer["text"])], answer["answer_start"]

In [ ]:
from tqdm import tqdm
class VnCoreSquad(SquadProcessor):
  train_file = "train-v1.1.json"
  dev_file = "dev-v1.1.json"
  def _create_examples(self, input_data, set_type):
        is_training = set_type == "train"
        examples = []
        demso=0
        for entry in tqdm(input_data):
            title = entry["title"] 
            for paragraph in entry["paragraphs"]:
                context_text = VnCoreNLP1(paragraph["context"])
                for qa in paragraph["qas"]:
                    qas_id = qa["id"]
                    question_text = VnCoreNLP1(qa["question"])
                    start_position_character = None
                    answer_text = None
                    answers = []

                    is_impossible = qa.get("is_impossible", False)
                    if not is_impossible:
                        if is_training:
                            answer = qa["answers"][0]
                            answer_text, start_position_character=checkAns(answer, context_text)
                            qa["answers"][0].update({"text": answer_text})
                            qa["answers"][0].update({"answer_start": start_position_character})
                        else:
                            for k in range(0, len(qa["answers"])):
                              ans, start_position=checkTempAns(qa["answers"][k],context_text)
                              qa["answers"][k].update({"text": ans})
                              qa["answers"][k].update({"answer_start": start_position})
                              
                            answers = qa["answers"]
                    example = SquadExample(
                        qas_id=qas_id,
                        question_text=question_text,
                        context_text=context_text,
                        answer_text=answer_text,
                        start_position_character=start_position_character,
                        title=title,
                        is_impossible=is_impossible,
                        answers=answers,
                    )
                    examples.append(example)
        return examples
processor = VnCoreSquad()

In [ ]:
train_examples = processor.get_train_examples('','train_ViQuAD.json')
dev_examples = processor.get_dev_examples('','dev_ViQuAD.json')

100%|██████████| 18/18 [00:11<00:00,  1.50it/s]


In [ ]:
from transformers.data.processors.squad import squad_convert_examples_to_features

In [ ]:
train_features, train_dataset = squad_convert_examples_to_features(train_examples, 
                                                       tokenizer, 
                                                       max_seq_length = 256, 
                                                       doc_stride = 81,
                                                       max_query_length = 81,
                                                       is_training = True,
                                                       return_dataset = 'pt',
                                                       threads = 10
                                                       )

convert squad examples to features:   0%|          | 0/18579 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
add example index and unique id: 100%|██████████| 18579/18579 [00:00<00:00, 696109.46it/s]


In [ ]:
del train_examples

In [ ]:
dev_features, dev_dataset = squad_convert_examples_to_features(dev_examples, 
                                                       tokenizer, 
                                                       max_seq_length = 256, 
                                                       doc_stride = 81,
                                                       max_query_length = 81,
                                                       is_training = False,
                                                       return_dataset = 'pt',
                                                       threads = 10
                                                       )

convert squad examples to features:   0%|          | 0/2285 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
add example index and unique id: 100%|██████████| 2285/2285 [00:00<00:00, 602955.94it/s]


In [ ]:
def to_list(tensor):
    return tensor.detach().cpu().tolist()

In [ ]:
import os
def evaluate(model, tokenizer, dev_dataset, dev_examples, dev_features):
    eval_sampler = SequentialSampler(dev_dataset)
    eval_dataloader = DataLoader(dev_dataset, sampler=eval_sampler, batch_size=32)
    all_results = []
#     start_time = timeit.default_timer()
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(device) for t in batch)
        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
#                "token_type_ids": batch[2],
            }
            example_indices = batch[3]
            outputs = model(**inputs)
        for i, example_index in enumerate(example_indices):
            eval_feature = dev_features[example_index.item()]
            unique_id = int(eval_feature.unique_id)
            output = [to_list(output[i]) for output in outputs]
            if len(output) >= 5:
                start_logits = output[0]
                start_top_index = output[1]
                end_logits = output[2]
                end_top_index = output[3]
                cls_logits = output[4]

                result = SquadResult(
                    unique_id,
                    start_logits,
                    end_logits,
                    start_top_index=start_top_index,
                    end_top_index=end_top_index,
                    cls_logits=cls_logits,
                )
            else:
                start_logits, end_logits = output
                result = SquadResult(unique_id, start_logits, end_logits)
            all_results.append(result)
    
    output_prediction_file = os.path.join("./", "predictions_{}.json".format(""))
    output_nbest_file = os.path.join("./", "nbest_predictions_{}.json".format(""))
    output_null_log_odds_file = os.path.join("./", "null_odds_{}.json".format(""))
    predictions = compute_predictions_logits(
            dev_examples,
            dev_features,
            all_results,
            20,
            128,
            False,
            output_prediction_file,
            output_nbest_file,
            output_null_log_odds_file,
            True,
            False,
            0.0,
            tokenizer,
        )
    results = squad_evaluate(dev_examples, predictions)
    return results

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import trange, tqdm
device = torch.device('cuda')

In [ ]:
num_epochs = 3
tb_writer = SummaryWriter()
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=8)
t_total = len(train_dataloader) // 1 * num_epochs

output_dir = os.path.join('./', 'final_model')
no_decay = ["bias", "LayerNorm.weight"]

optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0,
    },
    {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, eps = 1e-8)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=814, num_training_steps=t_total
)

device = torch.device('cuda')

model.to(device)

global_step = 1
epochs_trained = 0
steps_trained_in_current_epoch = 0
tr_loss, logging_loss = 0.0, 0.0

model.zero_grad()
train_iterator = trange(
    epochs_trained, int(num_epochs), desc="Epoch", disable=-1 not in [-1, 0]
)

from functools import partial
tqdm = partial(tqdm, position=0, leave=True)
max_f1 = 0
patient = 0
for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=False)
    for step, batch in enumerate(epoch_iterator):
        model.train()
        batch = tuple(t.to(device) for t in batch)
        
        inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "start_positions": batch[3],
            "end_positions": batch[4],
        }
        
        outputs = model(**inputs)
        loss = outputs[0]
        loss.backward()
        tr_loss += loss.item()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        scheduler.step()
        model.zero_grad()
        global_step += 1

        if global_step % 5000 == 0:
            print(" global_step = %s, average loss = %s", global_step, tr_loss/global_step)
    model.eval()
    results = evaluate(model, tokenizer, dev_dataset, dev_examples, dev_features)
    print("exact: " + str(results['exact']))
    print(results['f1'])
    if results['f1'] >= max_f1:
        model_to_save = model.module if hasattr(model, "module") else model
        model_to_save.save_pretrained(output_dir)
        max_f1 = results['f1']
        patient = 0
    else:
        patient = patient+1
    if patient == 3:
        break

del model

Evaluating: 100%|██████████| 83/83 [01:05<00:00,  1.27it/s]


exact: 66.72519754170325
84.50685196857535


Iteration:  75%|███████▍  | 2138/2861 [27:41<09:21,  1.29it/s]

 global_step = %s, average loss = %s 5000 1.4784544331520795


Evaluating: 100%|██████████| 83/83 [01:05<00:00,  1.27it/s]


exact: 65.84723441615452
84.85232878663012


Evaluating: 100%|██████████| 83/83 [01:05<00:00,  1.27it/s]


exact: 66.85689201053556
85.50837192796267


Epoch: 100%|██████████| 3/3 [1:55:24<00:00, 2308.32s/it]


In [ ]:
model = RobertaForQuestionAnswering(phobert.config).from_pretrained("./final_model").to(device)
tokenizer_1 = AutoTokenizer.from_pretrained("vinai/phobert-large")

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
test_examples = processor.get_dev_examples('', 'test_ViQuAD.json')
test_features, test_dataset = squad_convert_examples_to_features(test_examples, 
                                                       tokenizer, 
                                                       max_seq_length = 256, 
                                                       doc_stride = 81,
                                                       max_query_length = 81,
                                                       is_training = False,
                                                       return_dataset = 'pt',
                                                       threads = 10
                                                       )

convert squad examples to features:   0%|          | 0/2210 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
add example index and unique id: 100%|██████████| 2210/2210 [00:00<00:00, 675957.98it/s]


In [ ]:
results = evaluate(model, tokenizer_1, test_dataset, test_examples, test_features)
for key, value in results.items():
    print(key, value)

Evaluating: 100%|██████████| 85/85 [01:07<00:00,  1.27it/s]


exact 63.31521739130435
f1 83.42177211251239
total 2208
HasAns_exact 63.31521739130435
HasAns_f1 83.42177211251239
HasAns_total 2208
best_exact 63.31521739130435
best_exact_thresh 0.0
best_f1 83.42177211251239
best_f1_thresh 0.0
